## CNN 

In [1]:
import numpy as np 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten 
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras.optimizers import Adam
from keras import regularizers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import pickle
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
import Functions as func
import var
import Viz
import cv2
from skimage.segmentation import mark_boundaries

Using plaidml.keras.backend backend.


In [2]:
### best is model weights 2

In [3]:
def get_conv_model_normal(dim):
    inp_shape = dim
    act = 'relu'
    drop = .25
    kernal_reg = regularizers.l1(.001)
    optimizer = Adam(lr = .001)
    
    model = Sequential() 
    
    model.add(Conv2D(64, kernel_size=(3,3),activation=act, input_shape = inp_shape, 
                     kernel_regularizer = kernal_reg,
                     kernel_initializer = 'he_uniform',  padding = 'same', name = 'Input_Layer'))
#     model.add(Dense(64, activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),  strides = (3,3)))
    
    
    model.add(Conv2D(64, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
#     model.add(Dense(64, activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))
    

    
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))

    
    model.add(Flatten())

    
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))

    model.add(Dropout(drop))

    model.add(Dense(3, activation='softmax', name = 'Output_Layer'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model 


In [9]:
def non_max_suppression(boxes, probs, overlapThresh=0.5):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []

    # if the bounding boxes are integers, convert them to floats -- this
    # is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    # initialize the list of picked indexes
    pick = []

    # grab the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # compute the area of the bounding boxes and grab the indexes to sort
    # (in the case that no probabilities are provided, simply sort on the
    # bottom-left y-coordinate)
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = y2

    # if probabilities are provided, sort on them instead
    if probs is not None:
        idxs = probs

    # sort the indexes
    idxs = np.argsort(idxs)
    # keep looping while some indexes still remain in the indexes list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the index value
        # to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of the bounding
        # box and the smallest (x, y) coordinates for the end of the bounding
        # box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]

        # delete all indexes from the index list that have overlap greater
        # than the provided overlap threshold
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))

    # return the indexes of only the bounding boxes to keep
    return pick

In [32]:
def non_max_suppression(boxes, probs, overlapThresh=0.6):


    # if the bounding boxes are integers, convert them to floats -- this
    # is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    # initialize the list of picked indexes
    pick = []

    # grab the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # compute the area of the bounding boxes and grab the indexes to sort
    # (in the case that no probabilities are provided, simply sort on the
    # bottom-left y-coordinate)
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = y2

    # if probabilities are provided, sort on them instead

    # sort the indexes
    idxs = np.argsort(idxs)
    # keep looping while some indexes still remain in the indexes list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the index value
        # to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of the bounding
        # box and the smallest (x, y) coordinates for the end of the bounding
        # box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]

        # delete all indexes from the index list that have overlap greater
        # than the provided overlap threshold
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
    
    
    prob_cat = [np.argmax(probs[i]) for i in pick]
    print(prob_cat)
    print(pick)
    new_pick = [i for idx, i in enumerate(pick) if prob_cat[idx] in [1,2]]
    print(new_pick)
    # return the indexes of only the bounding boxes to keep
    return pick

### Images

In [39]:
#WITH MAX SUPPRESION


#     clone = img.copy() 
#     clone2 = img.copy()
#     for idx, pred in enumerate(predictions): 
#         if pred[1] == 1 or pred[2] == 1: 
#             startx, starty, endx, endy = locations[idx]
#             cv2.rectangle(clone, (startx, starty), (endx, endy), (0,0,255), 2)
            
#     no_sum = predictions[:,0].sum()
#     pistol_sum = predictions[:,1].sum()
#     rifle_sum = predictions[:,2].sum() 
#     sum_array = np.array([no_sum, pistol_sum, rifle_sum])
#     print(sum_array)
#     clone = img.copy()
#     clone2 = img.copy()
#     print(np.argmax(sum_array))
#     cat_predictions = predictions[:,np.argmax(sum_array)]
#     pred_max_idx = np.argmax(cat_predictions)
#     pred_max = cat_predictions[pred_max_idx]
    
#     pred_max_window = locations[pred_max_idx]
#     startx, starty, endx, endy = pred_max_window


#     cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,0,255),2)
#     cat = category_dict[np.argmax(sum_array)]
#     text = f'{cat}'
#     cv2.putText(clone, text, (startx, starty), cv2.FONT_HERSHEY_SIMPLEX, .45, (0,0,255),2)

def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
    return edged

def get_edged(img, dim): 
    blurred = cv2.GaussianBlur(img, (3,3), 0)
    wide = cv2.Canny(blurred, 10,200)
    tight = cv2.Canny(blurred, 225, 250)
    auto = auto_canny(blurred)
    wide = cv2.resize(wide, dim, interpolation = cv2.INTER_CUBIC)
    tight = cv2.resize(tight, dim, interpolation = cv2.INTER_CUBIC)
    auto = cv2.resize(auto, dim, interpolation = cv2.INTER_CUBIC)
    return wide


def get_img_prediction_bounding_box(path, model, dim, edge = False):
    img = func.get_image_value(path, dim, edge = edge)

    if edge == True:
        img = img.reshape(1, img.shape[0], img.shape[1], 1)
    else: 
        img = img.reshape(1, img.shape[0], img.shape[1], 3)
    pred = model.predict(img)[0]
    
    category_dict = {0: 'No Weapon', 1: 'Handgun', 2: 'Rifle'}
    cat_index = np.argmax(pred)
    cat = category_dict[cat_index]
    
    print(f'{path}\t\t{cat_index}||{cat}\t\t{pred.max()}\t\t{pred}')
    
    img = cv2.imread(path)
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()
    rects = ss.process() 

    windows = []
    locations = []
    for x, y, w,h in rects: 
        startx = x 
        starty = y 
        endx = x+w 
        endy = y+h 
        roi = img[starty:endy, startx:endx]
        if edge == True:
            roi = get_edged(roi, dim = dim)
        roi = cv2.resize(roi, dsize =dim, interpolation = cv2.INTER_CUBIC)
        windows.append(roi)
        locations.append((startx, starty, endx, endy))

    windows = np.array(windows)
    print(windows.shape)
    if edge == True:
        windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 1)
    else: 
        windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 3)
    windows = np.array(windows)
    locations = np.array(locations)
    predictions = model.predict(windows)
    
#     pick = non_max_suppression(locations, probs = predictions)
#     clone = img.copy() 
#     clone2 = img.copy()
#     for idx in pick: 
#         startx, startx, endx, endy = locations[idx]
#         cv2.rectangle(clone, (startx, starty), (endx, endy), (0,0,255), 2)
    clone = img.copy() 
    clone2 = img.copy()
    for idx, i in enumerate(predictions): 
        if np.argmax(i) in [1,2]: 
            startx, starty, endx, endy = locations[idx]
            cv2.rectangle(clone, (startx, starty), (endx, endy), (0,0,255), 2)
        
    
#GOES HERE 
    cv2.imshow(f'Test', np.hstack([clone, clone2]))
    cv2.waitKey(0)
    ss.clear()


    return predictions


base_path = '../Tests/Photos'
edge = False
if edge == True: 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 1)
else: 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 3)
    
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('../models/CNN-ModelCheckpointWeights1.h5')

img = get_img_prediction_bounding_box(f'{base_path}/NonGun.jpeg', normal_model, var.norm_dimension, edge = edge)

# for file in os.listdir(base_path):
#     img = get_img_prediction_bounding_box(f'{base_path}/{file}', normal_model, var.norm_dimension, edge = edge)
#     break
    
    

../Tests/Photos/NonGun.jpeg		0||No Weapon		0.9999905824661255		[9.999906e-01 8.386117e-07 8.615097e-06]
(3544, 150, 150, 3)


In [36]:
for idx, i in enumerate(img): 
    if np.argmax(i) in [1,2]: 
        print(idx, i)

3 [0. 1. 0.]
6 [0. 1. 0.]
9 [1.94709e-37 0.00000e+00 1.00000e+00]
10 [0. 1. 0.]
11 [0. 1. 0.]
12 [0. 1. 0.]
13 [0. 1. 0.]
17 [0. 1. 0.]
18 [0. 1. 0.]
19 [0. 1. 0.]
22 [0. 1. 0.]
24 [0. 1. 0.]
27 [0. 1. 0.]
30 [0. 1. 0.]
31 [0.0000000e+00 1.0000000e+00 2.1840914e-27]
33 [0. 1. 0.]
35 [0. 1. 0.]
42 [3.2808984e-36 0.0000000e+00 1.0000000e+00]
50 [0. 1. 0.]
51 [0. 1. 0.]
52 [0. 0. 1.]
56 [0. 1. 0.]
57 [0. 1. 0.]
58 [0. 1. 0.]
61 [1.2171935e-09 1.0000000e+00 0.0000000e+00]
62 [0. 1. 0.]
69 [0. 1. 0.]
71 [0. 1. 0.]
78 [0. 1. 0.]
79 [0. 1. 0.]
80 [0. 0. 1.]
81 [0. 1. 0.]
84 [0. 1. 0.]
92 [0. 1. 0.]
93 [0. 0. 1.]
97 [0. 1. 0.]
98 [0. 1. 0.]
102 [2.7614758e-10 1.0000000e+00 0.0000000e+00]
105 [0.00530338 0.9946967  0.        ]
106 [0. 1. 0.]
112 [0. 1. 0.]
114 [0. 1. 0.]
122 [7.0983995e-28 1.0000000e+00 0.0000000e+00]
126 [0. 1. 0.]
127 [0. 1. 0.]
130 [0. 1. 0.]
134 [1.669844e-22 1.000000e+00 0.000000e+00]
136 [7.075996e-27 1.000000e+00 0.000000e+00]
138 [0. 1. 0.]
143 [0. 1. 0.]
145 [0. 1. 0.]

### Video 

In [5]:
from tqdm import tqdm
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
    return edged

def get_edged(img, dim): 
    blurred = cv2.GaussianBlur(img, (3,3), 0)
    wide = cv2.Canny(blurred, 10,200)
    tight = cv2.Canny(blurred, 225, 250)
    auto = auto_canny(blurred)
    wide = cv2.resize(wide, dim, interpolation = cv2.INTER_CUBIC)
    tight = cv2.resize(tight, dim, interpolation = cv2.INTER_CUBIC)
    auto = cv2.resize(auto, dim, interpolation = cv2.INTER_CUBIC)
    return wide

def get_vid_prediction_bounding_box(path, model, dim, edge = False): 
    vid = cv2.VideoCapture(path)
    total_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    pbar = tqdm(total = total_frames, desc = f'Splitting Video Into {total_frames} Frames')
    images = [] 
    sucess =1 
    while True: 
        try:
            success, img = vid.read() 
            img = cv2.resize(img, dim, interpolation = cv2.INTER_CUBIC)
            images.append(img)
            pbar.update(1)
        except: 
            break
        

    pbar.close()
    images = np.array(images)
    
    return images
   
edge= True 
if edge == True: 
    dim = (var.norm_dimension[0],var.norm_dimension[1], 1)
else: 
    dim = (var.norm_dimension[0], var.norm_dimension[1], 3)
    
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('../models/CNN-ModelCheckpointWeights2.h5')

images = get_vid_prediction_bounding_box('../Tests/Videos/Pistol2.mp4', normal_model, var.norm_dimension, edge = True)
# predictions = normal_model.predict(images)

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"
Splitting Video Into 720 Frames: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 720/720 [00:02<00:00, 311.66it/s]


In [6]:
def window_prob_func(img, model, dim, edge):
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()
    rects = ss.process() 

    windows = []
    locations = []
    for x, y, w,h in rects: 
        startx = x 
        starty = y 
        endx = x+w 
        endy = y+h 
        roi = img[starty:endy, startx:endx]
        if edge == True:
            roi = get_edged(roi, dim = dim)
        roi = cv2.resize(roi, dsize =dim, interpolation = cv2.INTER_CUBIC)
        windows.append(roi)
        locations.append((startx, starty, endx, endy))

    windows = np.array(windows)
    if edge == True:
        windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 1)
    else: 
        windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 3)
    predictions = model.predict(windows)
    locations = np.array(locations)
#     for idx, pred in enumerate(predictions): 
#         if pred[1] == 1 or pred[2] == 1: 
#             startx, starty, endx, endy = locations[idx]
#             return (1, (startx, starty, endx, endy))
    pick = non_max_suppression(locations, probs = None)
    return [predictions[i] for i in pick]
    for idx in pick: 
        prob = predictions[idx]
        startx, startx, endx, endy = locations[idx]
        return (prob, (startx, starty, endx, endy))

        
    return None

pbar = tqdm(images, desc= 'Getting Base Prediction and Extracting Sliding Window... Sit Back, This Will Take A While')
windows_prob = [window_prob_func(img, normal_model, var.norm_dimension, edge = edge) for img in pbar]



Getting Base Prediction and Extracting Sliding Window... Sit Back, This Will Take A While: 100%|████████████████████████████████████████████████████████| 720/720 [05:14<00:00,  2.29it/s]


In [8]:
for i in windows_prob: 
    print([np.argmax(y) for y in i])

[1]
[2, 2, 1, 1]
[1, 1, 1, 1]
[2, 2, 2, 1, 1]
[1]
[2, 2]
[1, 1, 0]
[1, 1, 1]
[1, 1, 1]
[1, 1]
[1, 1]
[0]
[2, 0]
[1]
[0, 1, 1, 1]
[0, 1]
[0, 1, 1]
[0, 1, 1]
[0, 1]
[2, 1, 1, 1, 1, 2]
[2, 1]
[2, 1]
[1, 1]
[0, 1, 1]
[2, 1, 1]
[1, 1, 1]
[1, 1, 1, 1, 2, 1]
[1, 1, 1]
[1, 1, 1, 2]
[1, 1, 1]
[1, 2, 0, 1]
[1, 1, 1, 1, 1]
[1, 0]
[0, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1, 1]
[1, 1, 0]
[2, 1, 1, 0]
[1, 1, 1]
[1, 0]
[1, 1]
[1, 1, 1]
[1, 1, 0]
[1, 0]
[1, 1, 2, 1, 1, 0]
[1, 0]
[1, 1, 1, 1, 1, 2]
[0, 1, 1]
[1, 1, 1]
[2, 1, 1, 1, 1, 1]
[1, 1, 1, 1]
[0, 1, 1, 1, 1]
[1, 0, 1]
[1, 1, 1, 0, 1, 1]
[1, 2, 1, 1, 1]
[1, 1, 1, 1, 1, 0]
[0, 1, 1, 1, 1, 1]
[1, 0, 1]
[1, 1, 0]
[0]
[1, 1, 1, 0]
[0, 1]
[0, 1]
[1, 1]
[2, 1]
[1, 1]
[0, 1]
[0, 1, 1]
[0, 1, 1]
[1, 1, 1]
[1, 0]
[1, 2]
[1, 1]
[0]
[1, 1, 1, 1]
[2, 1]
[1]
[1]
[1, 1]
[1, 1, 1, 1]
[2]
[1]
[0, 1, 2]
[1, 0]
[1, 1, 1, 1, 1]
[1, 1, 1, 1]
[1]
[0]
[1, 2, 1, 1]
[1, 1, 1, 0]
[1, 1, 0]
[1, 0]
[1, 1, 1]
[2, 1, 0]
[2, 2, 1]
[1]
[2, 1]
[1]
[1, 1, 1, 1, 1, 1]
[1]
[2, 1, 0, 1

[0, 1, 1, 1]
[0]
[0, 1, 1]
[0]
[1, 1, 0]
[0, 2, 1, 2]
[2, 1, 0]
[0]
[0]
[0]
[0]
[2, 0, 2, 1, 1]
[0, 2, 2, 2]
[0, 2, 2, 2, 1]
[0, 1, 1, 2, 1]
[1, 1, 1, 1, 1, 1]
[0, 1]
[2, 0]
[0]
[0]
[0, 1]
[0, 1]
[0]
[1, 2, 2, 2]
[0, 1]
[0, 1]
[2, 1, 0, 1]
[0, 2, 2, 2, 2]
[0, 1, 1, 1]
[1, 0]
[1, 0, 1]
[0, 2, 2]
[0, 1]
[0, 1, 1]
[1, 1, 1, 1]
[0, 1, 1]
[0]
[0, 0]
[0]
[0]
[0]
[0]
[0, 2, 1, 1, 1]
[1, 0]
[0, 0]
[1, 1, 1, 2, 1]
[0]
[0, 1, 2]
[1]


In [8]:
last = pickle.load(open('../Pickles/WindowsPropPistol_CNNTester.p', 'rb'))
for i in last: 
    print(i)

None
None
None
None
None
(1, (95, 3, 124, 28))
None
None
(1, (86, 55, 125, 130))
None
None
None
None
(1, (70, 92, 141, 150))
None
None
(1, (71, 82, 141, 150))
None
None
None
None
None
None
None
None
None
(1, (43, 0, 76, 91))
None
(1, (16, 29, 56, 137))
None
(1, (63, 40, 125, 106))
None
None
(1, (80, 52, 108, 87))
(1, (80, 51, 109, 87))
(1, (80, 51, 109, 87))
(1, (80, 51, 109, 87))
None
(1, (80, 51, 108, 87))
(1, (54, 51, 104, 102))
None
None
None
None
(1, (43, 51, 111, 105))
None
None
(1, (50, 99, 70, 150))
(1, (50, 68, 78, 150))
None
(1, (51, 71, 77, 150))
None
(1, (99, 123, 135, 150))
None
None
None
(1, (78, 84, 123, 122))
None
(1, (53, 74, 79, 118))
None
None
None
None
None
None
None
None
None
(1, (83, 49, 133, 87))
None
(1, (82, 54, 150, 135))
(1, (80, 86, 101, 144))
None
(1, (79, 92, 101, 143))
None
None
None
None
None
None
None
None
None
(1, (58, 126, 134, 150))
None
None
None
None
None
None
None
None
None
None
None
None
(1, (71, 93, 108, 132))
None
None
None
(1, (0, 0, 150, 150)

(1, (85, 0, 150, 93))
(1, (56, 0, 80, 60))
(1, (111, 102, 122, 150))
(1, (0, 0, 60, 82))
None
None
None
None
None
(1, (106, 31, 150, 83))
None
None
None
None
None
None
None
None
(1, (37, 131, 82, 136))
None
None
None
None
None
None
None
(1, (87, 90, 143, 150))
None
None
(1, (84, 85, 114, 139))
None
(1, (102, 87, 115, 122))
(1, (93, 8, 150, 33))
None
None
None
None
None
None
None
(1, (43, 0, 119, 81))
None
None
None
None
None
(1, (17, 89, 72, 131))
None
None
None
(1, (95, 74, 150, 150))
None
None
None
None


In [ ]:
pickle.dump(windows_prob, open('../Pickles/WindowsPropPistol_CNNTester.p', 'wb'))

vid_dim = (224,224)
vid_dim = var.norm_dimension
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('../Tests/VideoTests/TestCNNTESTER.mp4',fourcc, 10, vid_dim)


for prob, img in zip(windows_prob, images): 
    clone = img.copy()
    if prob == None:
#         clone = cv2.resize(clone, dsize =vid_dim, interpolation = cv2.INTER_CUBIC)      
        out.write(clone)
    else:
        (p, (startx, starty, endx, endy)) = prob
        cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,0,255),2)
        out.write(clone)
        
out.release()
cv2.destroyAllWindows()

In [ ]:
#WITHOUT MAX SUPPRESION
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
    return edged

def get_edged(img, dim): 
    blurred = cv2.GaussianBlur(img, (3,3), 0)
    wide = cv2.Canny(blurred, 10,200)
    tight = cv2.Canny(blurred, 225, 250)
    auto = auto_canny(blurred)
    wide = cv2.resize(wide, dim, interpolation = cv2.INTER_CUBIC)
    tight = cv2.resize(tight, dim, interpolation = cv2.INTER_CUBIC)
    auto = cv2.resize(auto, dim, interpolation = cv2.INTER_CUBIC)
    return wide



def get_img_prediction_bounding_box(path, model, dim, edge = False):
    img = func.get_image_value(path, dim, edge = edge)

    if edge == True:
        img = img.reshape(1, img.shape[0], img.shape[1], 1)
    else: 
        img = img.reshape(1, img.shape[0], img.shape[1], 3)
    pred = model.predict(img)[0]
    
    category_dict = {0: 'No Weapon', 1: 'Handgun', 2: 'Rifle'}
    cat_index = np.argmax(pred)
    cat = category_dict[cat_index]
    
    print(f'{path}\t\t{cat_index}||{cat}\t\t{pred.max()}\t\t{pred}')
    
    
    img = cv2.imread(path)
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()
    rects = ss.process() 

    windows = []
    locations = []
    for x, y, w,h in rects: 
        startx = x 
        starty = y 
        endx = x+w 
        endy = y+h 
        roi = img[starty:endy, startx:endx]
        if edge == True:
            roi = get_edged(roi, dim = dim)
        roi = cv2.resize(roi, dsize =dim, interpolation = cv2.INTER_CUBIC)
        windows.append(roi)
        locations.append((startx, starty, endx, endy))

    windows = np.array(windows)
    print(windows.shape)
    if edge == True:
        windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 1)
    else: 
        windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 3)
    predictions = model.predict(windows)
    
    clone = img.copy() 
    clone2 = img.copy()
    for idx, pred in enumerate(predictions): 
        if pred[1] == 1 or pred[2] == 1: 
            startx, starty, endx, endy = locations[idx]
            cv2.rectangle(clone, (startx, starty), (endx, endy), (0,0,255), 2)
            break
            
    no_sum = predictions[:,0].sum()
    pistol_sum = predictions[:,1].sum()
    rifle_sum = predictions[:,2].sum() 
    sum_array = np.array([no_sum, pistol_sum, rifle_sum])
    print(sum_array)
#     clone = img.copy()
#     clone2 = img.copy()
#     print(np.argmax(sum_array))
#     cat_predictions = predictions[:,np.argmax(sum_array)]
#     pred_max_idx = np.argmax(cat_predictions)
#     pred_max = cat_predictions[pred_max_idx]
    
#     pred_max_window = locations[pred_max_idx]
#     startx, starty, endx, endy = pred_max_window


#     cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,0,255),2)
#     cat = category_dict[np.argmax(sum_array)]
#     text = f'{cat}'
#     cv2.putText(clone, text, (startx, starty), cv2.FONT_HERSHEY_SIMPLEX, .45, (0,0,255),2)
   

    cv2.imshow(f'Test', np.hstack([clone, clone2]))
    cv2.waitKey(0)
    ss.clear()


    return predictions
base_path = '../Tests/Photos'
edge = True
if edge == True: 
    dim = (x_train.shape[1], x_train.shape[2], 1)
else: 
    dim = (x_train.shape[1], x_train.shape[2], 3)
    
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('../models/CNN-ModelCheckpointWeights2.h5')

for file in os.listdir(base_path): 
    img = get_img_prediction_bounding_box(f'{base_path}/{file}', normal_model, var.norm_dimension, edge = edge)
    
    

In [ ]:
img

In [ ]:
base_path = '../Tests/Photos'
edge = True
if edge == True: 
    dim = (x_train.shape[1], x_train.shape[2], 1)
else: 
    dim = (x_train.shape[1], x_train.shape[2], 3)
    
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('../models/CNN-ModelCheckpointWeights2.h5')


for file in os.listdir(base_path): 
    if file == '.ipynb_checkpoints':
        continue
    full_path = f'{base_path}/{file}'
    if edge == True: 
        img = func.get_image_value(full_path, var.norm_dimension, edge = True)   
        img = img.reshape(1, img.shape[0], img.shape[1], 1)
    else:
        img = func.get_image_value(full_path, var.norm_dimension) 
        print(img.shape)
        img = img.reshape(1, img.shape[0], img.shape[1], 3)
    pred = normal_model.predict(img)[0]
    print(f'{file}\t\t{np.argmax(pred)}\t\t{pred.max()}\t\t{pred}')

In [ ]:
base_path = '../Tests/Photos'
edge = False
if edge == True: 
    dim = (x_train.shape[1], x_train.shape[2], 1)
else: 
    dim = (x_train.shape[1], x_train.shape[2], 3)
    
normal_model = get_conv_model_normal(dim)
normal_model.load_weights('../models/CNN-ModelCheckpointWeights1.h5')


for file in os.listdir(base_path): 
    if file == '.ipynb_checkpoints':
        continue
    full_path = f'{base_path}/{file}'
    if edge == True: 
        img = func.get_image_value(full_path, var.norm_dimension, edge = True)   
        img = img.reshape(1, img.shape[0], img.shape[1], 1)
    else:
        img = func.get_image_value(full_path, var.norm_dimension) 
        print(img.shape)
        img = img.reshape(1, img.shape[0], img.shape[1], 3)
    pred = normal_model.predict(img)[0]
    print(f'{file}\t\t{np.argmax(pred)}\t\t{pred.max()}\t\t{pred}')

In [ ]:
img = get_img_prediction_bounding_box('../Tests/Photos/Pistol.jpg', normal_model, var.norm_dimension)

In [ ]:
assert False

In [ ]:
from lime import lime_image

model 3

if edge == True: 
    img = func.get_image_value(full_path, var.norm_dimension, edge = True)   
    img = np.stack((img,)*3, axis = -1)
else:
    img = func.get_image_value(full_path, var.norm_dimension)    
    img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])

explainer = lime_image.LimeImageExplainer()

explanation = explainer.explain_instance(img, normal_model.predict, top_labels = 5, hide_color = 0, 
                                         num_samples = 2000)

temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only = False,
                                           num_features = 10, hide_rest = False)
plt.imshow(mark_boundaries(temp/2 + .5, mask))